In [1]:
import pandas as pd
from pandas_ods_reader import read_ods
import numpy as np
import datetime as datetime
import re

# Datenvorverarbeitung

In [2]:
path = "../data/Arbeitsplanung_steuerung_Wickelmaschinen_alt.ods"
sheet_idx = 14
df = read_ods(path, sheet_idx)
df = df.rename(columns=df.iloc[10])
df = df.rename(columns={df.columns[0]:'Nichts'})
df = df.drop('Nichts', axis=1)
df = df.drop(np.arange(12))
df = df.reset_index(drop=True)
df = df[['lfd. Nr.', 'Fertigungsauf-tragsnummer', 'Teilenummer', 'Bezeichnung', 'Rohrtyp', 'Nenn-durchmesser',
         'Auftragsmenge', 'Artikellänge', 'Bandüberlappung Funktionslage', 'Bandüberlappung Beilaufpapier',               'Lagenvariante', 'Nummer Wickel-rohrmaschine', 'Maschinen-gruppe', 'Werkzeug-nummer',                           'Rüstzeit für WKZ/Materialwechsel', 'Rüstzeit für Coilwechsel', 'Maschinen-laufzeit',                            'Fertigungszeit pro Mengeneinheit']]
df.head()

,lfd. Nr.,Fertigungsauf-tragsnummer,Teilenummer,Bezeichnung,Rohrtyp,Nenn-durchmesser,Auftragsmenge,Auftragsmenge,Artikellänge,Artikellänge,Bandüberlappung Funktionslage,Bandüberlappung Beilaufpapier,Lagenvariante,Nummer Wickel-rohrmaschine,Maschinen-gruppe,Werkzeug-nummer,Rüstzeit für WKZ/Materialwechsel,Rüstzeit für Coilwechsel,Maschinen-laufzeit,Fertigungszeit pro Mengeneinheit
0,1.0,92021412719.0,60PAP2060,Aluminium – Papier – Kunststoff,APK,75.0,1000.0,0.0,1000.0,0.0,2.0,10.0,Variante 1,Wickelrohrmaschine 1531,1.0,A0 023,15.0,2.0,323.96,0.31
1,2.0,92021529395.0,70AA30000,Aluminium – Aluminium,AA,70.0,3000.0,34.0,900.0,30.0,2.0,10.0,Variante 1,Wickelrohrmaschine 1531,1.0,A0 023,15.0,2.0,147.19,3.89
2,3.0,92021529919.0,80PAK,Papier – Aluminium – Kunststoff,PAK,80.0,3000.0,24.0,900.0,25.0,2.0,10.0,Variante 1,Wickelrohrmaschine 1531,1.0,A0 023,15.0,2.0,124.49,4.56
3,4.0,92021530000.0,90APK004,Aluminium – Papier – Kunststoff,APK,90.0,3000.0,20.0,900.0,25.0,2.0,10.0,Variante 3,Wickelrohrmaschine 1531,1.0,A0 023,15.0,2.0,104.03,4.45
4,5.0,92021539118.0,55PAP3985,Papier – Aluminium – Papier,PAP,55.0,480.0,20.0,3985.0,25.0,2.0,10.0,Variante 1,Wickelrohrmaschine 1535,3.0,A0 023,15.0,2.0,570.94,1.16


In [4]:
def parse_machine_number(df, column):
    """
    Function extracts the machine number of a expression in the format
    "Wickelrohrmaschine 1234" and gives back only the machines with a
    number expression.

    Parameters
    ----------
    df : pd.DataFrame
        dataframe with a column which contains a string expression
         for the machine
    column: string
        Name of the column that contains information about the machine

    Returns
    ---------
    pd.DataFrame
        Updated dataframe with new columns 'machine' and 'machine_id
    """
    #df = df[df.Nummer Wickel-rohrmaschine.str.startswith('Wickelrohrmaschine')]
    df['machine_id'] = df.apply(lambda row: str(row[column][-4:]).strip(),
                             axis=1)
    return df


In [5]:
df = parse_machine_number(df, 'Nummer Wickel-rohrmaschine')
df.head()

,lfd. Nr.,Fertigungsauf-tragsnummer,Teilenummer,Bezeichnung,Rohrtyp,Nenn-durchmesser,Auftragsmenge,Auftragsmenge,Artikellänge,Artikellänge,...,Bandüberlappung Beilaufpapier,Lagenvariante,Nummer Wickel-rohrmaschine,Maschinen-gruppe,Werkzeug-nummer,Rüstzeit für WKZ/Materialwechsel,Rüstzeit für Coilwechsel,Maschinen-laufzeit,Fertigungszeit pro Mengeneinheit,machine_id
0,1.0,92021412719.0,60PAP2060,Aluminium – Papier – Kunststoff,APK,75.0,1000.0,0.0,1000.0,0.0,...,10.0,Variante 1,Wickelrohrmaschine 1531,1.0,A0 023,15.0,2.0,323.96,0.31,1531
1,2.0,92021529395.0,70AA30000,Aluminium – Aluminium,AA,70.0,3000.0,34.0,900.0,30.0,...,10.0,Variante 1,Wickelrohrmaschine 1531,1.0,A0 023,15.0,2.0,147.19,3.89,1531
2,3.0,92021529919.0,80PAK,Papier – Aluminium – Kunststoff,PAK,80.0,3000.0,24.0,900.0,25.0,...,10.0,Variante 1,Wickelrohrmaschine 1531,1.0,A0 023,15.0,2.0,124.49,4.56,1531
3,4.0,92021530000.0,90APK004,Aluminium – Papier – Kunststoff,APK,90.0,3000.0,20.0,900.0,25.0,...,10.0,Variante 3,Wickelrohrmaschine 1531,1.0,A0 023,15.0,2.0,104.03,4.45,1531
4,5.0,92021539118.0,55PAP3985,Papier – Aluminium – Papier,PAP,55.0,480.0,20.0,3985.0,25.0,...,10.0,Variante 1,Wickelrohrmaschine 1535,3.0,A0 023,15.0,2.0,570.94,1.16,1535


In [6]:
def calculate_timestamps(df, start):
    machines = df['machine_id'].astype(int).unique()
    df = df.assign(starttime=0)
    df = df.assign(endtime=0)
    # Für jede Maschine 
    for machine in machines:
        df_machine = df[df['machine_id'] == str(machine)]
        timestamp = start
        # Entsprechend der Reihenfolge timestamps berechnen
        for index, row in df_machine.iterrows():
            order_num = row['Fertigungsauf-tragsnummer']
            if timestamp.hour > 18: # Nächster Tag
                timestamp = datetime.datetime(timestamp.year, timestamp.month, timestamp.day+1, 6, 0, 0)
            df.loc[df['Fertigungsauf-tragsnummer'] == order_num, ['starttime']] = timestamp
            try:
                prod_time = int(row['Maschinen-laufzeit'])
            except:
                prod_time = 60
            timestamp = timestamp + datetime.timedelta(minutes=prod_time)
            order_num = row['Fertigungsauf-tragsnummer']
            df.loc[df['Fertigungsauf-tragsnummer'] == order_num, ['endtime']] = timestamp
    return df
            
df = calculate_timestamps(df, datetime.datetime(2022, 3, 16, 6, 0, 0))
df.head()

,lfd. Nr.,Fertigungsauf-tragsnummer,Teilenummer,Bezeichnung,Rohrtyp,Nenn-durchmesser,Auftragsmenge,Auftragsmenge,Artikellänge,Artikellänge,...,Nummer Wickel-rohrmaschine,Maschinen-gruppe,Werkzeug-nummer,Rüstzeit für WKZ/Materialwechsel,Rüstzeit für Coilwechsel,Maschinen-laufzeit,Fertigungszeit pro Mengeneinheit,machine_id,starttime,endtime
0,1.0,92021412719.0,60PAP2060,Aluminium – Papier – Kunststoff,APK,75.0,1000.0,0.0,1000.0,0.0,...,Wickelrohrmaschine 1531,1.0,A0 023,15.0,2.0,323.96,0.31,1531,2022-03-16 06:00:00,2022-03-16 11:23:00
1,2.0,92021529395.0,70AA30000,Aluminium – Aluminium,AA,70.0,3000.0,34.0,900.0,30.0,...,Wickelrohrmaschine 1531,1.0,A0 023,15.0,2.0,147.19,3.89,1531,2022-03-16 11:23:00,2022-03-16 13:50:00
2,3.0,92021529919.0,80PAK,Papier – Aluminium – Kunststoff,PAK,80.0,3000.0,24.0,900.0,25.0,...,Wickelrohrmaschine 1531,1.0,A0 023,15.0,2.0,124.49,4.56,1531,2022-03-16 13:50:00,2022-03-16 15:54:00
3,4.0,92021530000.0,90APK004,Aluminium – Papier – Kunststoff,APK,90.0,3000.0,20.0,900.0,25.0,...,Wickelrohrmaschine 1531,1.0,A0 023,15.0,2.0,104.03,4.45,1531,2022-03-16 15:54:00,2022-03-16 17:38:00
4,5.0,92021539118.0,55PAP3985,Papier – Aluminium – Papier,PAP,55.0,480.0,20.0,3985.0,25.0,...,Wickelrohrmaschine 1535,3.0,A0 023,15.0,2.0,570.94,1.16,1535,2022-03-16 06:00:00,2022-03-16 15:30:00


In [7]:
def preprocess_part_nr(df):
    #df['part'] = df.apply(lambda row: row['Teilenummer'][:row['Teilenummer'].rfind('[A-Z]')],axis=1)
    df['part'] = df.apply(lambda row: re.findall("[0-9]+[A-Z]+", row['Teilenummer'])[-1], axis=1)
    return df

df = preprocess_part_nr(df)
df.head()

,lfd. Nr.,Fertigungsauf-tragsnummer,Teilenummer,Bezeichnung,Rohrtyp,Nenn-durchmesser,Auftragsmenge,Auftragsmenge,Artikellänge,Artikellänge,...,Maschinen-gruppe,Werkzeug-nummer,Rüstzeit für WKZ/Materialwechsel,Rüstzeit für Coilwechsel,Maschinen-laufzeit,Fertigungszeit pro Mengeneinheit,machine_id,starttime,endtime,part
0,1.0,92021412719.0,60PAP2060,Aluminium – Papier – Kunststoff,APK,75.0,1000.0,0.0,1000.0,0.0,...,1.0,A0 023,15.0,2.0,323.96,0.31,1531,2022-03-16 06:00:00,2022-03-16 11:23:00,60PAP
1,2.0,92021529395.0,70AA30000,Aluminium – Aluminium,AA,70.0,3000.0,34.0,900.0,30.0,...,1.0,A0 023,15.0,2.0,147.19,3.89,1531,2022-03-16 11:23:00,2022-03-16 13:50:00,70AA
2,3.0,92021529919.0,80PAK,Papier – Aluminium – Kunststoff,PAK,80.0,3000.0,24.0,900.0,25.0,...,1.0,A0 023,15.0,2.0,124.49,4.56,1531,2022-03-16 13:50:00,2022-03-16 15:54:00,80PAK
3,4.0,92021530000.0,90APK004,Aluminium – Papier – Kunststoff,APK,90.0,3000.0,20.0,900.0,25.0,...,1.0,A0 023,15.0,2.0,104.03,4.45,1531,2022-03-16 15:54:00,2022-03-16 17:38:00,90APK
4,5.0,92021539118.0,55PAP3985,Papier – Aluminium – Papier,PAP,55.0,480.0,20.0,3985.0,25.0,...,3.0,A0 023,15.0,2.0,570.94,1.16,1535,2022-03-16 06:00:00,2022-03-16 15:30:00,55PAP


# Pm4py